In [25]:
import numpy as np
import pandas as pd
import os
import platform
from StringIO import StringIO

In [12]:
if platform.system() == 'Windows':
    !dir
else:
    !ls

 Volume in drive C is TI10709800F
 Volume Serial Number is 9CBB-1390

 Directory of C:\Users\Vasia\Documents\projects\smart_energy_hw1

11/19/2017  04:33 PM    <DIR>          .
11/19/2017  04:33 PM    <DIR>          ..
11/19/2017  04:21 PM               123 .gitignore
11/19/2017  04:29 PM    <DIR>          .ipynb_checkpoints
11/19/2017  03:20 PM       495,032,008 combined_data_1.txt
11/19/2017  03:21 PM       555,207,357 combined_data_2.txt
11/19/2017  03:21 PM       465,157,779 combined_data_3.txt
11/19/2017  03:21 PM       552,537,802 combined_data_4.txt
11/19/2017  03:21 PM           577,547 movie_titles.csv
11/19/2017  04:33 PM             1,141 prediction.ipynb
11/19/2017  03:21 PM        10,782,692 probe.txt
11/19/2017  03:21 PM        52,452,386 qualifying.txt
11/19/2017  03:20 PM             5,916 README
              10 File(s)  2,131,754,751 bytes
               3 Dir(s)  28,686,585,856 bytes free


In [51]:
movie_titles = pd.read_csv('movie_titles_cleaned.csv', skipinitialspace=True,
                           header=None,
                           index_col=0,
                           names=["year","title"],
                           dtype={"year":np.float32,"title":str}).fillna('')
